In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *

import sklearn 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

## 1 

Пусть $t$ — значение статистики критерия Стьюдента для независимых выборок, $nu$ — соответствующее число степеней свободы. Какая из строчек кода вычисляет достигаемый уровень значимости t-критерия при альтернативе $H_1\colon \mu_1>\mu_2H$

In [2]:
t = 1
nu = 9

stats.t.cdf(t, nu) # u1 < u2
1 - stats.t.cdf(t, nu) # u1 > u2
2 * (1 - stats.t.cdf(abs(t), nu)) # u1 <> u2

0.34343639613791366

**Ответ**: `1 - stats.t.cdf(t, nu)`

## 2 

Выберите ситуации, в которых допустимо проверять нулевую гипотезу $H_0\colon \mu_1=\mu_2H$ тив односторонней альтернативы $H_1\colon \mu_1>\mu_2H$

**Ответ**: 
* Признаки устроены так, что $ \mu_1<\mu_2 $ в принципе невозможно.

## 3
Выберите верные утверждения о проблеме Беренца-Фишера.

**Ответ**: 
* Приближённое решение проблемы Беренца-Фишера достаточно точно при $n_1=n_2$
* Проблема заключается в том, что не существует точного способа сравнить средние двух выборок с неизвестными дисперсиями

# 4 
Уровень кальция в крови здоровых молодых женщин равен в среднем 9.5 милиграммам на децилитр и имеет характерное стандартное отклонение 0.4 мг/дл. В сельской больнице Гватемалы для 160 здоровых беременных женщин при первом обращении для ведения беременности был измерен уровень кальция; среднее значение составило 9.57 мг/дл. Можно ли утверждать, что средний уровень кальция в этой популяции отличается от 9.5?

Посчитайте достигаемый уровень значимости. Поскольку известны только среднее и дисперсия, а не сама выборка, нельзя использовать стандартные функции критериев — нужно реализовать формулу достигаемого уровня значимости самостоятельно.

Округлите ответ до четырёх знаков после десятичной точки.

In [3]:
mu_0 = 9.5
std_0 = 0.4 # sig генеральной совокупности, тогда будем использовать z-критерий (и z-распределение)

n = 160
mu_2 = 9.57

In [4]:
z = (mu_2 - mu_0) / (std_0 / np.sqrt(n)); z

2.213594362117875

In [5]:
print(f"P-value: {round(2 * (1 - stats.norm.cdf(abs(z))), 4)}")

P-value: 0.0269


**Ответ**: 0.0269

## 5

Как вы считаете, это отличие в среднем уровне кальция в крови практически значимо?

**Ответ**: Скорее всего нет: стандартное отклонение уровня в популяции намного больше полученного различия между средними  

## 6

Имеются данные о стоимости и размерах 53940 бриллиантов: diamonds.txt

Отделите 25% случайных наблюдений в тестовую выборку с помощью функции `sklearn.cross_validation.train_test_split` (зафиксируйте `random state = 1`). На обучающей выборке настройте две регрессионные модели:

* линейную регрессию с помощью LinearRegression без параметров
* случайный лес из 10 деревьев с помощью RandomForestRegressor с `random_state=1`.

Какая из моделей лучше предсказывает цену бриллиантов? Сделайте предсказания на тестовой выборке, посчитайте модули отклонений предсказаний от истинных цен. Проверьте гипотезу об одинаковом среднем качестве предсказаний, вычислите достигаемый уровень значимости. Отвергается ли гипотеза об одинаковом качестве моделей против двусторонней альтернативы на уровне значимости $\alpha=0.05$?

In [6]:
diamonds = pd.read_csv('diamonds.txt', sep='\t')

In [7]:
diamonds.head()

,carat,depth,table,price,x,y,z
0,0.23,61.5,55.0,326,3.95,3.98,2.43
1,0.21,59.8,61.0,326,3.89,3.84,2.31
2,0.23,56.9,65.0,327,4.05,4.07,2.31
3,0.29,62.4,58.0,334,4.20,4.23,2.63
4,0.31,63.3,58.0,335,4.34,4.35,2.75


In [8]:
X = diamonds.drop(['price'], axis=1)
y = diamonds['price']

`sklearn.model_selection.train_test_split` — Split arrays or matrices into random train and test subsets.

In [9]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=.25, random_state=1)

In [10]:
len(y_test) / len(y)

0.25

### Линейная регрессия

In [11]:
lr = LinearRegression()

In [12]:
lr.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [13]:
lr.score(X_test, y_test)

0.8615514393114848

In [14]:
abs(lr.predict(X_test) - np.array(y_test)) # модуль отклонений для лин. регрессии

array([ 214.42235621, 1016.46053685,   41.87871693, ..., 2171.49180326,
        103.82818096,  159.32529706])

### Случайный лес

In [15]:
rf = RandomForestRegressor(n_estimators=10, random_state=0)

In [16]:
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=0, verbose=0,
                      warm_start=False)

In [17]:
rf.score(X_test, y_test)

0.871244269286064

In [18]:
abs(rf.predict(X_test) - np.array(y_test)) # модуль отклонений для дерева

array([ 184.85833333,  395.8       ,  416.3       , ..., 1255.1       ,
          4.3       ,  279.46666667])

Для проверки гипотезы об одинаковом среднем качестве предсказаний используем t-критерий для связанных выборок, так как методы ML применяются для одной выборки.

$H_0$ Разности средних абсолютных ошибок равны

$H_1$ Не равны

In [19]:
t_statistic, p_val = stats.ttest_rel(abs(lr.predict(X_test) - np.array(y_test)), abs(rf.predict(X_test) - np.array(y_test)))
p_val

9.005140089959077e-33

In [20]:
p_val < 0.05

True

**Ответ**: Отвергается, $p < \alpha$

## 7

В предыдущей задаче посчитайте 95% доверительный интервал для разности средних абсолютных ошибок предсказаний регрессии и случайного леса. Чему равна его ближайшая к нулю граница? Округлите до десятков (поскольку случайный лес может давать немного разные предсказания в зависимости от версий библиотек, мы просим вас так сильно округлить, чтобы полученное значение наверняка совпало с нашим).

In [21]:
DescrStatsW(abs(lr.predict(X_test) - np.array(y_test)) - abs(rf.predict(X_test) - np.array(y_test))).tconfint_mean()

(68.28216277577089, 95.06579713171071)

In [22]:
int(round(68.28216277577089, -1))

70

**Ответ**: 70